In [1]:
import os
os.getcwd()

'e:\\Bappy-ML-US\\Text-Summerization-Project\\research'

In [2]:
change_dir = "e:\\Bappy-ML-US\\Text-Summerization-Project"
os.chdir(change_dir)
os.getcwd()

'e:\\Bappy-ML-US\\Text-Summerization-Project'

In [3]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: str

In [4]:
from textSummerizer.constants import *
from textSummerizer.utils.common import read_yaml, creat_directories

In [5]:
class ConfigurationManager():
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_filepath)
        self.config = read_yaml(params_filepath)
        
        creat_directories([self.config.artifacts_root])


    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        creat_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir = config.root_dir,
            data_path = config.data_path,
            tokenizer_name = config.tokenizer_name
        )

        return data_transformation_config


In [7]:
import os
from textSummerizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset, load_from_disk

In [8]:

class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)


    
    def convert_examples_to_features(self,example_batch):
        input_encodings = self.tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )
        
        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['summary'], max_length = 128, truncation = True )
            
        return {
            'input_ids' : input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }
    

    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched = True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir,"samsum_dataset"))


In [9]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e

[2024-07-18 11:04:07,698: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-07-18 11:04:07,700: INFO: common: yaml file: params.yaml loaded successfully]


BoxKeyError: "'ConfigBox' object has no attribute 'artifacts_root'"

In [10]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: str

from textSummerizer.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from textSummerizer.utils.common import read_yaml, creat_directories

class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH, params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        
        # Merge params into config if needed
        self.config.update(self.params)
        
        # Handle the missing key with a default value or error message
        self.artifacts_root = self.config.get('artifacts_root', 'default_artifacts_root')

        creat_directories([self.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config['data_transformation']

        creat_directories([config['root_dir']])

        data_transformation_config = DataTransformationConfig(
            root_dir=Path(config['root_dir']),
            data_path=Path(config['data_path']),
            tokenizer_name=config['tokenizer_name']
        )

        return data_transformation_config

import os
from textSummerizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_from_disk

class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)

    def convert_examples_to_features(self, example_batch):
        input_encodings = self.tokenizer(example_batch['dialogue'], max_length=1024, truncation=True)
        
        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['summary'], max_length=128, truncation=True)
            
        return {
            'input_ids': input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }

    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched=True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir, "samsum_dataset"))

try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.convert()
except Exception as e:
    logger.error(f"An error occurred: {e}")
    raise e


[2024-07-18 11:04:37,683: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-07-18 11:04:37,683: INFO: common: yaml file: params.yaml loaded successfully]
[2024-07-18 11:04:37,697: INFO: common: created directory at: artifacts]
[2024-07-18 11:04:37,699: INFO: common: created directory at: artifacts/data_transformation]


c:\Users\zishan\.conda\envs\textSummerization\lib\site-packages\huggingface_hub\file_download.py:157: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\zishan\.cache\huggingface\hub\models--google--pegasus-cnn_dailymail. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
Map:   0%|          | 0/14732 [00:00<?, ? examples/s]c:\Users\zishan\.conda\envs\textSumm